https://www.analyticsvidhya.com/blog/2020/04/vehicle-detection-opencv-python/

In [1]:
import os
import cv2
import numpy as np

In [2]:
pathIn = "/home/priyanshjain/Downloads/Stride_Assignment/obstacle_input/"
pathOut = "obstacle_output/"

In [17]:
def video_to_frames(input_loc, output_loc):

    cap = cv2.VideoCapture(input_loc)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imwrite(output_loc + "%03d" % count + '.jpg', frame)
        count = count + 1

        if (count > (video_length-1)):
            cap.release()
            break

def frames_to_video(input_loc,output_loc):
    frame_array = []
    files = os.listdir(input_loc)  
    files.sort()

    for i,file_name in enumerate(files):
        img = cv2.imread(input_loc+file_name)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)

    out = cv2.VideoWriter(output_loc,cv2.VideoWriter_fourcc(*'DIVX'), 29.97, size)

    for j in range(len(frame_array)):
        out.write(frame_array[j])
    out.release()

In [7]:
def load_input_frames(path):
    img = []
    images = os.listdir(path)
    images.sort()

    for i, image_name in enumerate(images):
        image = cv2.imread(path+image_name,1)
        img.append(image)
    
    return img

def image_preprocess(A, B):
    grayA = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(B, cv2.COLOR_BGR2GRAY)
    diff_image = cv2.absdiff(grayB, grayA)
    blur = cv2.GaussianBlur(diff_image, (5,5), 0)
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((5,5),np.uint8)
    dilated_car = cv2.dilate(thresh,kernel,iterations = 6)
    dilated_train = cv2.dilate(thresh,kernel,iterations = 3)
    return dilated_car, dilated_train

def detect(dilated_car, dilated_train):
    contours_car, hierarchy_car = cv2.findContours(dilated_car.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contours_train, hierarchy_train = cv2.findContours(dilated_train.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    rect_car = []
    rect_train = []
    
    for i,cntr in enumerate(contours_car):
        a = cv2.boundingRect(cntr)
        if (a[0] + a[1] > 550) & (a[1] > 50) & (cv2.contourArea(cntr) > 1500) & (a[1] - 0.84*a[0] + 605 > 0):
            rect_car.append(a)

    for i,cntr in enumerate(contours_train):
        a = cv2.boundingRect(cntr)
        if (a[1]+0.5*a[3] - 0.84*(a[0]+0.5*a[2]) + 650 < 0) & (cv2.contourArea(cntr) > 1500):
            rect_train.append(cntr)

    return rect_car, rect_train    
    
def display_detection(img,idx,points_car, points_train, path):
    plag = img.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    if (len(points_car) != 0):
        for i,coor in enumerate(points_car):
            x,y,w,h = coor
            cv2.rectangle(plag, (x,y),(x+w,y+h), (0,0,255), 2)

    if (len(points_train) == 1):
        x,y,w,h = cv2.boundingRect(points_train[0])
        cv2.rectangle(plag, (x,y),(x+w,y+h), (255,0,0), 2)

    cv2.imwrite(path+"%03d" % idx+'.png',plag)

    return plag
    
def write_output_frames(input_path, output_path):
    input = load_input_frames(input_path)
    
    for i in range(0,len(input)-1):
        processed_car, processed_train = image_preprocess(input[i],input[i+1])
        detection_car, detection_train = detect(processed_car, processed_train)
        display_detection(input[i],i,detection_car, detection_train,output_path)

In [11]:
def detect_obstacle_in_video():
    video_to_frames('Obstacle.mp4', pathIn)
    write_output_frames(pathIn,pathOut)
    frames_to_video(pathOut, 'obstacle_detection.mp4')

In [18]:
detect_obstacle_in_video()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
